# Pitchfork Scraper

## Import Libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

## Get code for reviews page

In [2]:
url = "https://pitchfork.com/reviews/albums/"
r = requests.get(url)
soup = BeautifulSoup(r.content)

## Scrape info for each review

In [5]:
table = soup.findAll('div', attrs = {'class':"SummaryItemWrapper-iwvBff gVhZsz summary-item summary-item--has-border summary-item--no-icon summary-item--text-align-left summary-item--layout-placement-text-below summary-item--layout-position-image-left summary-item--layout-proportions-50-50 summary-item--side-by-side-align-top summary-item--side-by-side-image-right-mobile-false summary-item--standard SummaryCollectionGridSummaryItem-WColm dzwmya"}) 

df = pd.DataFrame(columns = ['url', 'album_name', 'album_image', 'artist', 'genres', 'rating', 'tagline'])
df_created = False

counter = 10

for i in table:
    
    # Album Name
    album_name = i.find('h3', attrs = {'class': 'SummaryItemHedBase-hiFYpQ jwYeiM summary-item__hed'}).get_text()

    # Image URL
    album_cover = i.find('img', attrs = {'class': 'ResponsiveImageContainer-eybHBd fptoWY responsive-image__image'}, src=True)['src']

    # Artist Name
    try:
        artist = i.findAll('div', attrs = {'class': 'SummaryItemSubHedBase-gMyBBg bijetA summary-item__sub-hed'})[0].get_text()
    except:
        artist = "Various Artists"

    # Print Genres
    try:
        genre_helper = i.findAll('div', attrs = {'class':'RubricWrapper-dKmCNX cStFUw rubric SummaryItemRubric-dguGKN lapGFj summary-item__rubric'})
        genres = genre_helper[0].findAll('a')[0].findAll('span')[0].get_text()
    except:
        genres = "Not Found"

    # Get URL
    hyperlink = i.find('a', attrs = {'SummaryItemHedLink-civMjp PNQqc summary-item-tracking__hed-link summary-item__hed-link summary-item__hed-link--underline-disable'}, href=True)
    url = "https://pitchfork.com" + hyperlink['href']

    # Send Request for review page
    time.sleep(1)
    review_soup = BeautifulSoup(requests.get(url).content)

    # Get Rating
    try:
        rating = review_soup.find('p', attrs = {'class': 'BaseWrap-sc-gjQpdd BaseText-ewhhUZ Rating-bkjebD iUEiRd bwCcXY imqiqZ'}).get_text()
    except:
        try:
            rating = review_soup.find('div', attrs = {'class': 'ScoreCircle-jAxRuP akdGf'}).get_text()
        except:
            rating = "NOT FOUND"

    # Get Summary
    try:
        tagline = review_soup.find('div', attrs = {'class': 'BaseWrap-sc-gjQpdd BaseText-ewhhUZ SplitScreenContentHeaderDekDown-csTFQR iUEiRd Byyns MVQMg'}).get_text()
    except:
        try:
            tagline = review_soup.find('div', attrs = {'class': 'BaseWrap-sc-gjQpdd BaseText-ewhhUZ MultiReviewContentHeaderDek-dQARIe iUEiRd Byyns bBVtRE'}).get_text()
        except:
            tagline = "NOT FOUND"

    review_info = {
        'url': [url],
        'album_name': [album_name],
        'album_image': [album_cover],
        'artist': [artist],
        'genres': [genres],
        'rating': [rating],
        'tagline': [tagline]
    }

    df = pd.concat([df, pd.DataFrame(review_info)], ignore_index=True)

    counter = counter - 1
    if counter == 0:
        break

## View DataFrame

In [ ]:
df

'https://pitchfork.com/reviews/albums/autre-ne-veut-love-guess-who-xxooxx/'